reference: https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/06_CIFAR-10.ipynb

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os

## Load the data

In [3]:
import cifar10

In [3]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [4]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [7]:
X_val=images_train[:5000,:,:,:]
y_onehot_val=labels_train[:5000,:]
X_train=images_train[5000:,:,:,:]
y_onehot=labels_train[5000:,:]

In [5]:
# evaluate performance on some data 
def perf_eval(logit_pred, y_true):
    """a function to evaluate performance of predicted y values vs true class labels"""
    # now look at some data
    print('    sample pred: {0}\n    sample true: {1}'.format(np.argmax(logit_pred[0:20],1),np.argmax(y_true[0:20],1)))
    # avg accuracy
    is_correct_vals = np.equal(np.argmax(logit_pred,1),np.argmax(y_true,1))
    #accuracy_vals = np.mean(is_correct_vals)
    #print('    mean classification accuracy: {0}%'.format(100*accuracy_vals))
    # Dig in a little deeper.  Where did we make correct predictions?  Does this seem reasonable?
    print('    correct predictions by class: {0}'.format(y_true[is_correct_vals,:].sum(axis=0)))

In [6]:
# cnn conv stuff
def conv(x, W):
    """simple wrapper for tf.nn.conv2d"""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool(x):
    """simple wrapper for tf.nn.max_pool with stride size 2"""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def norm(x): 
    """simple wrapper for tf.nn.lrn... See section 3.3 of Krizhevsky 2012 for details"""
    return tf.nn.lrn(x, depth_radius=5, bias=2, alpha=1e-4, beta=0.75)

In [29]:
# elaborate the compute_logits code to include a variety of models
def compute_logits(x, model_type, pkeep):
    """Compute the logits of the model"""
    if model_type=='lr':
        W = tf.get_variable('W', shape=[32*32*3, 10])
        b = tf.get_variable('b', shape=[10])
        logits = tf.add(tf.matmul(x, W), b, name='logits_lr')
    elif model_type=='cnn_cf':
        # try a 1 layer cnn
        n1 = 64
        x_image = tf.reshape(x, [-1,32,32,3]) # batch, then width, height, channels
        # cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[5, 5, 3, n1])
        b_conv1 = tf.get_variable('b_conv1', shape=[n1])
        h_conv1 = tf.nn.relu(tf.add(conv(x_image, W_conv1), b_conv1))
        # fc layer to logits
        h_conv1_flat = tf.reshape(h_conv1, [-1, 32*32*n1])
        W_fc1 = tf.get_variable('W_fc1', shape=[32*32*n1, 10])
        b_fc1 = tf.get_variable('b_fc1', shape=[10])
        logits = tf.add(tf.matmul(h_conv1_flat, W_fc1), b_fc1, name='logits_cnn_cf')
    elif model_type=='cnn_cnf':
        # try a 1 layer cnn with a normalization layer
        n1 = 64
        x_image = tf.reshape(x, [-1,32,32,3]) # batch, then width, height, channels
        # cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[5, 5, 3, n1])
        b_conv1 = tf.get_variable('b_conv1', shape=[n1])
        h_conv1 = tf.nn.relu(tf.add(conv(x_image, W_conv1), b_conv1))
        # norm layer 1
        h_norm1 = norm(h_conv1)
        # fc layer to logits
        h_flat = tf.reshape(h_norm1, [-1, 32*32*n1])
        W_fc1 = tf.get_variable('W_fc1', shape=[32*32*n1, 10])
        b_fc1 = tf.get_variable('b_fc1', shape=[10])
        logits = tf.add(tf.matmul(h_flat, W_fc1), b_fc1, name='logits_cnn_cnf')     
    elif model_type=='cnn_cpncpnff':
        # 2 layer cnn
        n1 = 32
        n2 = 64
        n3 = 1024
        x_image = tf.reshape(x, [-1,32,32,3]) # batch, then width, height, channels
        # cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[5, 5, 3, n1])
        b_conv1 = tf.get_variable('b_conv1', shape=[n1])
        h_conv1 = tf.nn.relu(tf.add(conv(x_image, W_conv1), b_conv1))
        # pool 1
        h_pool1 = maxpool(h_conv1)
        # norm 1
        h_norm1 = norm(h_pool1)
        # cnn layer 2
        W_conv2 = tf.get_variable('W_conv2', shape=[5, 5, n1, n2])
        b_conv2 = tf.get_variable('b_conv2', shape=[n2])
        h_conv2 = tf.nn.relu(tf.add(conv(h_norm1, W_conv2), b_conv2))
        # pool 2
        h_pool2 = maxpool(h_conv2)
        # norm 2
        h_norm2 = norm(h_pool2)
        # fc layer to logits (8x8 since 2 rounds of maxpool)
        h_norm2_flat = tf.reshape(h_norm2, [-1, 8*8*n2])
        W_fc1 = tf.get_variable('W_fc1', shape=[8*8*n2, n3])
        b_fc1 = tf.get_variable('b_fc1', shape=[n3])
        h_fc1 = tf.nn.relu(tf.add(tf.matmul(h_norm2_flat, W_fc1), b_fc1))
        # one more fc layer
        # ... again, this is the logistic layer with softmax readout
        W_fc2 = tf.get_variable('W_fc2', shape=[n3,10])
        b_fc2 = tf.get_variable('b_fc2', shape=[10])
        logits = tf.add(tf.matmul(h_fc1, W_fc2), b_fc2, name='logits_cnn_cpncpnff')
    elif model_type=='cnn_cpncpnfdf':
        # same as above but add dropout.
        # 2 layer cnn
        n1 = 32
        n2 = 64
        n3 = 1024
        x_image = tf.reshape(x, [-1,32,32,3]) # batch, then width, height, channels
        # cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[5, 5, 3, n1])
        b_conv1 = tf.get_variable('b_conv1', shape=[n1])
        h_conv1 = tf.nn.relu(tf.add(conv(x_image, W_conv1), b_conv1))
        # pool 1
        h_pool1 = maxpool(h_conv1)
        # norm 1
        h_norm1 = norm(h_pool1)
        # cnn layer 2
        W_conv2 = tf.get_variable('W_conv2', shape=[5, 5, n1, n2])
        b_conv2 = tf.get_variable('b_conv2', shape=[n2])
        h_conv2 = tf.nn.relu(tf.add(conv(h_norm1, W_conv2), b_conv2))
        # pool 2
        h_pool2 = maxpool(h_conv2)
        # norm 2
        h_norm2 = norm(h_pool2)
        # fc layer to logits (8x8 since 2 rounds of maxpool)
        h_norm2_flat = tf.reshape(h_norm2, [-1, 8*8*n2])
        W_fc1 = tf.get_variable('W_fc1', shape=[8*8*n2, n3])
        b_fc1 = tf.get_variable('b_fc1', shape=[n3])
        h_fc1 = tf.nn.relu(tf.add(tf.matmul(h_norm2_flat, W_fc1), b_fc1))
        # insert a dropout layer here.
        h_fc1_drop = tf.nn.dropout(h_fc1, pkeep)
        # one more fc layer
        # ... again, this is the logistic layer with softmax readout
        W_fc2 = tf.get_variable('W_fc2', shape=[n3,10])
        b_fc2 = tf.get_variable('b_fc2', shape=[10])
        logits = tf.add(tf.matmul(h_fc1_drop, W_fc2), b_fc2, name='logits_cnn_cpncpnfdf')
    elif model_type=='vgg16':
        x = tf.reshape(x, [-1,32,32,3])
        #layer1
        #cnn layer 1.1
        W_conv11 = tf.get_variable('W_conv11', shape=[3, 3, 3, 64])
        b_conv11 = tf.get_variable('b_conv11', shape=[64])
        print(W_conv11,b_conv11)
        h_conv11 = tf.nn.relu(tf.add(conv(x, W_conv11), b_conv11))  #[batch,32,32,64]
        print(h_conv11)
        #cnn layer 1.2
        W_conv12 = tf.get_variable('W_conv12', shape=[3, 3, 64, 64])
        b_conv12 = tf.get_variable('b_conv12', shape=[64])
        h_conv12 = tf.nn.relu(tf.add(conv(h_conv11, W_conv12), b_conv12))
        #pool 1
        pool1=maxpool(h_conv12)

        #cnn layer 2.1
        W_conv21 = tf.get_variable('W_conv21', shape=[3, 3, 64, 128])
        b_conv21 = tf.get_variable('b_conv21', shape=[128])
        h_conv21 = tf.nn.relu(tf.add(conv(pool1, W_conv21), b_conv21))  
        #cnn layer 2.2
        W_conv22 = tf.get_variable('W_conv22', shape=[3, 3, 128, 128])
        b_conv22 = tf.get_variable('b_conv22', shape=[128])
        h_conv22 = tf.nn.relu(tf.add(conv(h_conv21, W_conv22), b_conv22))
        #pool 2
        pool2=maxpool(h_conv22)

        #cnn layer 3.1
        W_conv31 = tf.get_variable('W_conv31', shape=[3, 3, 128, 256])
        b_conv31 = tf.get_variable('b_conv31', shape=[256])
        h_conv31 = tf.nn.relu(tf.add(conv(pool2, W_conv31), b_conv31))  
        #cnn layer 3.2
        W_conv32 = tf.get_variable('W_conv32', shape=[3, 3, 256, 256])
        b_conv32 = tf.get_variable('b_conv32', shape=[256])
        h_conv32 = tf.nn.relu(tf.add(conv(h_conv31, W_conv32), b_conv32))
        #cnn layer 3.3
        W_conv33 = tf.get_variable('W_conv33', shape=[3, 3, 256, 256])
        b_conv33 = tf.get_variable('b_conv33', shape=[256])
        h_conv33 = tf.nn.relu(tf.add(conv(h_conv32, W_conv33), b_conv33))
        #pool 3
        pool3=maxpool(h_conv33)

        #cnn layer 4.1
        W_conv41 = tf.get_variable('W_conv41', shape=[3, 3, 256, 512])
        b_conv41 = tf.get_variable('b_conv41', shape=[512])
        h_conv41 = tf.nn.relu(tf.add(conv(pool3, W_conv41), b_conv41))  
        #cnn layer 4.2
        W_conv42 = tf.get_variable('W_conv42', shape=[3, 3, 512, 512])
        b_conv42 = tf.get_variable('b_conv42', shape=[512])
        h_conv42 = tf.nn.relu(tf.add(conv(h_conv41, W_conv42), b_conv42))
        #cnn layer 4.3
        W_conv43 = tf.get_variable('W_conv43', shape=[3, 3, 512, 512])
        b_conv43 = tf.get_variable('b_conv43', shape=[512])
        h_conv43 = tf.nn.relu(tf.add(conv(h_conv42, W_conv43), b_conv43))
        #pool4
        pool4=maxpool(h_conv43)

        #cnn layer 5.1
        W_conv51 = tf.get_variable('W_conv51', shape=[3, 3, 512, 512])
        b_conv51 = tf.get_variable('b_conv51', shape=[512])
        h_conv51 = tf.nn.relu(tf.add(conv(pool4, W_conv51), b_conv51))  
        #cnn layer 5.2
        W_conv52 = tf.get_variable('W_conv52', shape=[3, 3, 512, 512])
        b_conv52 = tf.get_variable('b_conv52', shape=[512])
        h_conv52 = tf.nn.relu(tf.add(conv(h_conv51, W_conv52), b_conv52))
        #cnn layer 5.3
        W_conv53 = tf.get_variable('W_conv53', shape=[3, 3, 512, 512])
        b_conv53 = tf.get_variable('b_conv53', shape=[512])
        h_conv53 = tf.nn.relu(tf.add(conv(h_conv52, W_conv53), b_conv53))
        #pool 5
        pool5=maxpool(h_conv53)

        #fc1-4096
        shape=int(np.prod(pool5.get_shape()[1:]))
        pool5_flat=tf.reshape(pool5,[-1,shape])
        W_fc1 = tf.get_variable('W_fc1', shape=[shape,4096])
        b_fc1 = tf.get_variable('b_fc1', shape=[4096])
        h_fc1 = tf.nn.relu(tf.add(tf.matmul(pool5_flat, W_fc1), b_fc1))

        #fc2-4096
        W_fc2 = tf.get_variable('W_fc2', shape=[4096,4096])
        b_fc2 = tf.get_variable('b_fc2', shape=[4096])
        h_fc2 = tf.nn.relu(tf.add(tf.matmul(h_fc1, W_fc2), b_fc2))

        #fc3-1000
        W_fc3 = tf.get_variable('W_fc3', shape=[4096,1000])
        b_fc3 = tf.get_variable('b_fc3', shape=[1000])
        h_fc3 = tf.nn.relu(tf.add(tf.matmul(h_fc2, W_fc3), b_fc3))
        
        #from softmax to logits
        W_sf = tf.get_variable('W_sf', shape=[1000,10])
        b_sf = tf.get_variable('b_sf', shape=[10])
        logits = tf.add(tf.matmul(h_fc3, W_sf), b_sf, name='logits_vgg16')
    else: 
        print('error not a valid model type')
    print(logits)
    return logits

def compute_cross_entropy(logits, y):
    # Compute the average cross-entropy across all the examples.
    numerical_instability_example = 0
    if numerical_instability_example:
        y_pred = tf.nn.softmax(logits, name='y_pred') # the predicted probability for each example.
        cross_ent = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
    else:
        print(logits,y)
        sm_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits, name='cross_ent_terms')
        cross_ent = tf.reduce_mean(sm_ce, name='cross_ent')
    return cross_ent

def compute_accuracy(logits, y):
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy

# VGG

In [30]:
# choose case to run 
opt_method = 'sgd'
model_type =  'vgg16'
dir_name = 'logs/scratch04x/'
batch_size = 50
n=50000
n_val=5000

In [ ]:
with tf.Graph().as_default():
    # We build the model here as before
    x = tf.placeholder(tf.float32, [None, 32*32*3], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')
    pkeep = tf.placeholder(tf.float32, name='pkeep')
    
    with tf.name_scope('model'):
        logits = compute_logits(x, model_type, pkeep)
    with tf.name_scope('loss'):
        loss = compute_cross_entropy(logits=logits, y=y)
    with tf.name_scope('accuracy'):
        accuracy = compute_accuracy(logits, y)
    
    with tf.name_scope('opt'):
        if opt_method == 'sgd':
            opt = tf.train.GradientDescentOptimizer(0.5)
        elif opt_method == 'rms':
            opt = tf.train.RMSPropOptimizer(.001)
        elif opt_method == 'adam':
            opt = tf.train.AdamOptimizer(1e-4)
        train_step = opt.minimize(loss)
    
    with tf.name_scope('summaries'):
        # create summary for loss and accuracy
        tf.summary.scalar('loss', loss) 
        tf.summary.scalar('accuracy', accuracy)
        # create summary for logits
        tf.summary.histogram('logits', logits)
        # create summary for input image
        tf.summary.image('input', tf.reshape(x, [-1, 32, 32, 3]))
    
        summary_op = tf.summary.merge_all()
    
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
        summary_writer_train = tf.summary.FileWriter(dir_name+'/train', sess.graph)
        summary_writer_val = tf.summary.FileWriter(dir_name+'/val')
        
        sess.run(tf.global_variables_initializer())
    
        for i in range(301):
            batch = np.floor(np.random.rand(batch_size)*(n-n_val)).astype(int)
            X_batch = X_train[batch,:,:,:].reshape([batch_size,-1])
            y_batch = y_onehot[batch]

            # now run
            _ , summary = sess.run((train_step, summary_op),
                                      feed_dict={x: X_batch, y: y_batch, pkeep:0.85})
            
            # write the summary output to file
            if i%100==0:
                summary_writer_train.add_summary(summary, i)

            # print diagnostics
            if i%100 == 0:
                X_batch = X_train[0:1000,:,:,:].reshape([1000,-1])
                y_batch = y_onehot[0:1000]
                (train_error,train_logits) = sess.run((accuracy,logits), {x: X_batch, y: y_batch, pkeep:1.0})
                print("\rStep {0:3d}: training accuracy {1:0.4f}".format(i, train_error), flush=True)
                # further diagnostics
                perf_eval(train_logits, y_batch)
                
            if i%100 == 0:
                X_batch = X_val.reshape([n_val,-1])
                y_batch = y_onehot_val
                (val_error, summary) = sess.run((accuracy,summary_op), {x:X_batch, y:y_batch, pkeep:1.0})
                print("\rStep {0:3d}: val accuracy {1:0.4f}".format(i, val_error), flush=True)
                summary_writer_val.add_summary(summary, i)


<tf.Variable 'W_conv11:0' shape=(3, 3, 3, 64) dtype=float32_ref> <tf.Variable 'b_conv11:0' shape=(64,) dtype=float32_ref>
Tensor("model/Relu:0", shape=(?, 32, 32, 64), dtype=float32)
Tensor("model/logits_vgg16:0", shape=(?, 10), dtype=float32)
Tensor("model/logits_vgg16:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.1000
    sample pred: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [   0.    0.  100.    0.    0.    0.    0.    0.    0.    0.]


In [21]:
model_type =  'lr'
with tf.Graph().as_default():
    # We build the model here as before
    x = tf.placeholder(tf.float32, [None, 32*32*3], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')
    pkeep = tf.placeholder(tf.float32, name='pkeep')
    
    with tf.name_scope('model'):
        logits = compute_logits(x, model_type, pkeep)
    with tf.name_scope('loss'):
        loss = compute_cross_entropy(logits=logits, y=y)
    with tf.name_scope('accuracy'):
        accuracy = compute_accuracy(logits, y)
    
    with tf.name_scope('opt'):
        if opt_method == 'sgd':
            opt = tf.train.GradientDescentOptimizer(0.5)
        elif opt_method == 'rms':
            opt = tf.train.RMSPropOptimizer(.001)
        elif opt_method == 'adam':
            opt = tf.train.AdamOptimizer(1e-4)
        train_step = opt.minimize(loss)
    
    with tf.name_scope('summaries'):
        # create summary for loss and accuracy
        tf.summary.scalar('loss', loss) 
        tf.summary.scalar('accuracy', accuracy)
        # create summary for logits
        tf.summary.histogram('logits', logits)
        # create summary for input image
        tf.summary.image('input', tf.reshape(x, [-1, 32, 32, 3]))
    
        summary_op = tf.summary.merge_all()
    
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
        summary_writer_train = tf.summary.FileWriter(dir_name+'/train', sess.graph)
        summary_writer_val = tf.summary.FileWriter(dir_name+'/val')
        
        sess.run(tf.global_variables_initializer())
    
        for i in range(301):
            batch = np.floor(np.random.rand(batch_size)*(n-n_val)).astype(int)
            X_batch = X_train[batch,:,:,:].reshape([batch_size,-1])
            y_batch = y_onehot[batch]

            # now run
            _ , summary = sess.run((train_step, summary_op),
                                      feed_dict={x: X_batch, y: y_batch, pkeep:0.85})
            
            # write the summary output to file
            if i%100==0:
                summary_writer_train.add_summary(summary, i)

            # print diagnostics
            if i%100 == 0:
                X_batch = X_train[0:1000,:,:,:].reshape([1000,-1])
                y_batch = y_onehot[0:1000]
                (train_error,train_logits) = sess.run((accuracy,logits), {x: X_batch, y: y_batch, pkeep:1.0})
                print("\rStep {0:3d}: training accuracy {1:0.4f}".format(i, train_error), flush=True)
                # further diagnostics
                perf_eval(train_logits, y_batch)
                
            if i%100 == 0:
                X_batch = X_val.reshape([n_val,-1])
                y_batch = y_onehot_val
                (val_error, summary) = sess.run((accuracy,summary_op), {x:X_batch, y:y_batch, pkeep:1.0})
                print("\rStep {0:3d}: val accuracy {1:0.4f}".format(i, val_error), flush=True)
                summary_writer_val.add_summary(summary, i)


Tensor("model/logits_lr:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.1210
    sample pred: [0 0 0 0 0 4 0 4 0 0 0 0 0 0 0 0 0 4 0 0]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 100.    0.    0.    0.   21.    0.    0.    0.    0.    0.]
Step   0: val accuracy 0.1242
Step 100: training accuracy 0.2060
    sample pred: [7 7 1 5 5 5 5 5 9 9 0 9 1 1 1 7 5 5 1 7]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [  1.  52.   0.   0.   0.  58.   5.  72.   0.  18.]
Step 100: val accuracy 0.2030
Step 200: training accuracy 0.2200
    sample pred: [7 0 0 5 5 5 5 5 8 7 0 7 0 1 1 0 0 5 7 0]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 88.   6.   0.   0.   0.  57.  12.  55.   2.   0.]
Step 200: val accuracy 0.2176
Step 300: training accuracy 0.1640
    sample pred: [4 3 1 1 3 3 3 3 3 1 0 3 3 1 

In [ ]:
with tf.Graph().as_default():
    # We build the model here as before
    x = tf.placeholder(tf.float32, [None, 32*32*3], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')
    pkeep = tf.placeholder(tf.float32, name='pkeep')
    
    with tf.name_scope('model'):
        logits = compute_logits(x, model_type, pkeep)
    with tf.name_scope('loss'):
        loss = compute_cross_entropy(logits=logits, y=y)
    with tf.name_scope('accuracy'):
        accuracy = compute_accuracy(logits, y)
    
    with tf.name_scope('opt'):
        if opt_method == 'sgd':
            opt = tf.train.GradientDescentOptimizer(0.5)
        elif opt_method == 'rms':
            opt = tf.train.RMSPropOptimizer(.001)
        elif opt_method == 'adam':
            opt = tf.train.AdamOptimizer(1e-4)
        train_step = opt.minimize(loss)
    
    with tf.name_scope('summaries'):
        # create summary for loss and accuracy
        tf.summary.scalar('loss', loss) 
        tf.summary.scalar('accuracy', accuracy)
        # create summary for logits
        tf.summary.histogram('logits', logits)
        # create summary for input image
        tf.summary.image('input', tf.reshape(x, [-1, 32, 32, 3]))
    
        summary_op = tf.summary.merge_all()
    
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
        summary_writer_train = tf.summary.FileWriter(dir_name+'/train', sess.graph)
        summary_writer_val = tf.summary.FileWriter(dir_name+'/val')
        
        sess.run(tf.global_variables_initializer())
    
        for i in range(20001):
            batch = np.floor(np.random.rand(batch_size)*(n-n_val)).astype(int)
            X_batch = X_train[batch,:,:,:].reshape([batch_size,-1])
            y_batch = y_onehot[batch]

            # now run
            _ , summary = sess.run((train_step, summary_op),
                                      feed_dict={x: X_batch, y: y_batch, pkeep:0.85})
            
            # write the summary output to file
            if i%100==0:
                summary_writer_train.add_summary(summary, i)

            # print diagnostics
            if i%100 == 0:
                X_batch = X_train[0:1000,:,:,:].reshape([1000,-1])
                y_batch = y_onehot[0:1000]
                (train_error,train_logits) = sess.run((accuracy,logits), {x: X_batch, y: y_batch, pkeep:1.0})
                print("\rStep {0:3d}: training accuracy {1:0.4f}".format(i, train_error), flush=True)
                # further diagnostics
                perf_eval(train_logits, y_batch)
                
            if i%100 == 0:
                X_batch = X_val.reshape([n_val,-1])
                y_batch = y_onehot_val
                (val_error, summary) = sess.run((accuracy,summary_op), {x:X_batch, y:y_batch, pkeep:1.0})
                print("\rStep {0:3d}: val accuracy {1:0.4f}".format(i, val_error), flush=True)
                summary_writer_val.add_summary(summary, i)


Step   0: training accuracy 0.1150
    sample pred: [9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [   0.    0.    0.    0.    0.    0.    0.    0.    0.  115.]
